In [ ]:
"""
okitamark
llamacpp GGUF
"""

## モデルとログファイルをセットする
MODEL_NAME = '/home/users/model/DataPilot-ArrowPro-7B-KUJIRA-Q8_0.gguf' ## モデルの指定
# LOG_FILE = __file__.replace('.py', '.log') ## デフォルトログファイル名, jupyterの時はNone
LOG_FILE = None ## ファイル名 or None


"""
Argument, Logging
"""
import argparse
import logging

## 引数の処理
## jupyterの場合コメント
# parser = argparse.ArgumentParser(description='OkitaMark')
# parser.add_argument('--model', type=str, default=MODEL_NAME)
# parser.add_argument('--log', type=str, default=LOG_FILE)
# args = parser.parse_args()
# MODEL_NAME = args.model
# LOG_FILE = args.log

## LOG_FILEが指定されていたら、ログファイルも出力する
if LOG_FILE is not None:
    handlers=[logging.FileHandler(LOG_FILE, mode='w'), logging.StreamHandler()]
else:
    handlers=[logging.StreamHandler()]

## Logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s', handlers=handlers) ## Message Only
LOG = logging.getLogger(__name__)


LOG.info('MODEL_NAME: ' + MODEL_NAME)
if LOG_FILE is not None: LOG.info('LOG_FILE:' + LOG_FILE)



"""
Base
"""
import os, time
import subprocess, argparse
## Warning非表示
import warnings
warnings.simplefilter('ignore')

## Logging
# import logging
# logging.basicConfig(level=logging.DEBUG, format='%(message)s') ## Message Only
# LOG = logging.getLogger(__name__)

## Util ##
## GPU Info
def gpu_info():
    gpucmd = 'nvidia-smi --query-gpu=name --format=csv,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU device: ' + gpuinfo.decode()

def gpu_mem():
    gpucmd = 'nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used --format=csv,,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU memory: ' + gpuinfo.decode()



"""
モデルの定義
llama-cpp
"""
from llama_cpp import Llama

model_name = MODEL_NAME
## init Model
llm = Llama(
    model_path=model_name,
    n_ctx = 4095, ## Context Length
    n_gpu_layers = -1, ## GPU Mem: -1=ALL, 0=None, N=n_layer
    use_mlock = False, ## システムがモデルを RAM に保持するように強制する
    chat_format = None,  ## create_chat_completion を呼び出すときに使用するチャット形式を指定する文字列
    verbose = True, ## 詳細出力の出力
)


## 推論
def predict(prompt, max_token=4095, temperature=0.0001, top_p=0.0001, seed=0, title=''):
    LOG.info('**'+title+'************************************************')
    LOG.info('**'+model_name)
    LOG.info('**'+gpu_info().rstrip('\n'))
    LOG.info('**'+gpu_mem().rstrip('\n'))
    LOG.info('**input:')
    LOG.info(prompt)
    LOG.info('**output:')
    
    ##　推論開始
    stime = time.perf_counter() ## 計測開始
    output = llm.create_completion(
        prompt,
        max_tokens = max_token,
        seed = seed,
        temperature = temperature,
        top_p = top_p,
        # top_k = 100
        # min_p = ,
        # typical_p = ,
        # frequency_penalty = 0,
        # presence_penalty = 0,
        # repeat_penalty = 1.1,
        # tfs_z = 1,
        # mirostat_mode =  0,
        # mirostat_tau = 5,
        # mirostat_eta = 0.1,
        # stop=["Instruction:", "Input:", "Response:", "\n"],
        # echo=True,
        # stop=[
        #     "ASSISTANT:",
        #     "USER:",
        #     "SYSTEM:",
        #     "### Human",
        #     "### Assistant",
        # ],
        # stream=True,
    )
    tm = time.perf_counter() - stime ## 計測終了
    
    ## output
    out = output["choices"][0]["text"]
    n_token_input = output['usage']['prompt_tokens']
    n_token_output = output['usage']['completion_tokens']
    n_token_total = output['usage']['total_tokens']

    ## No Streaming
    ## streamer=streamer をコメントすること
    LOG.info(out)
    # End

    # ## Stream
    # for out in output:
    #     s = out["choices"][0]["text"]
    #     print(s, end='')
            
    ## 計測結果
    LOG.info('**Result: %s, Time: %.2f, Input: %d, Output: %d, Total: %d, Token/sec: %.1f' % (title, tm, n_token_input, n_token_output, n_token_total, n_token_output/tm)) ## 終了時間, 出力文字数    
    LOG.info('**Result: %s, %s' % (title, gpu_mem().rstrip('\n')))
    LOG.info('\n\n')



"""
プロンプトテンプレート
モデルに該当したプロンプトを指定する
"""

####
## Vicuna1.5
def qa_vicuna(input,
      system='A chat between a human and an assistant.' ## システムプロンプト
      ):
    return """{s}
### Human: {i}
### Assistant: """.format(s=system, i=input)

####
## llama2, elayza, stablelm
def qa_llama2(input,
      system='あなたは誠実で優秀な日本人のアシスタントです' ## システムプロンプト
      ):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    return "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token='<s>', ##tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{system}{E_SYS}",
        prompt=input,
        e_inst=E_INST,
    )

####
## llama3
def qa_llama3(input,
      system='あなたは日本語を話すAIアシスタントです。日本語で回答してください。you MUST write Japanese language.' ## システムプロンプト
      ):
    return """<|start_header_id|>system<|end_header_id|>\n\n{s}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{i}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n""".format(s=system, i=input)

####
## 東工大LLM/swallow
def qa_swallow_chat(input,
      system='以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。' ## システムプロンプト
      ):
    return """{s}
    
### 指示:
{i}

### 応答:
""".format(s=system, i=input)

####
## Gemma
def qa_gemma(input,
      system='' ## システムプロンプト
      ):
    return """<start_of_turn>user
{i}<end_of_turn>
""".format(s=system, i=input)

####
## ArrowPro
## pad_token_id=tokenizer.eos_token_id
def qa_arrowpro(user_query):
    sys_msg = "あなたは日本語を話す優秀なアシスタントです。回答には必ず日本語で答えてください。"
    template = """[INST] <<SYS>>
{}
<</SYS>>

{}[/INST]"""
    return template.format(sys_msg,user_query)
    

## set Prompt Template
qa = qa_arrowpro ## モデルに該当したプロンプトを指定する

## for Test
predict(qa("""自己紹介をしてください。"""), title='自己紹介')


LOG.info('fin.')